# 06 => Cifar-10. 30 에폭 학습시켜보기. val_accuracy의 변화 추이 알아보기


=> 이후에는 CIFAR-10, CIFAR-100, ImageNet dataset에 대해 validation accuracy 줄어들지 않을 때까지 몇 epoch 학습시켜야하며, 시간 얼마나 소요되는지 실험 진행해야함.

[Reference] https://pytorch.org/tutorials/intermediate/ddp_tutorial.html

# Main

In [1]:
import os
import logging
from easydict import EasyDict
import numpy as np
import random

import time
import datetime

from deap import tools

In [2]:
from utils_kyy.utils_graph import make_random_graph
from utils_kyy.create_toolbox import create_toolbox_for_NSGA_RWNN

## 1. generation pool 구성하기 (Small RWNN 대상)

In [3]:
# 실험을 위한 환경 셋팅
run_code = 'test_kyy_CIFAR10_time_check'
stage_pool_path = './graph_pool' + '/' + run_code + '/'
log_path = './logs/' + run_code + '/'

# check & make the directory
if not os.path.exists(stage_pool_path): os.makedirs(stage_pool_path)
if not os.path.isdir(log_path): os.makedirs(log_path)

# write the log
log_file_name = log_path + 'logging.log'
logging.basicConfig(filename=log_file_name, level=logging.INFO)
logging.info('Start to write log.')

In [4]:
# make random graph pool
num_graph = 100
make_random_graph(num_graph, stage_pool_path)

######################################################
# => 최종적으로, num_graph와 stage_pool_path 를 인수로 받아서, 해당 path에 num_graph 수 만큼의 그래프 떨궈주는 함수 만들기
#    일단은 정해진 graph_model은 'WS', K, P 는 인수로 받지 말고 구현
#      =>  이후에 확장하기.
######################################################

Start to make random graph pool...
Finished


## 2. Main NSGA_RWNN

In [5]:
# define 'args_train' for evaluation
args_train = EasyDict({
    'lr_mode': 'cosine',
    'warmup_mode': 'linear',    # default
    'base_lr': 0.1,
    'momentum': 0.9, 
    'weight_decay': 0.00005,
    'print_freq': 100,

    'epochs': 30,
    'batch_size': 256,   # 128 => 256

    'workers': 32,  # 2 => 

    'warmup_epochs': 0,
    'warmup_lr': 0.0,
    'targetlr': 0.0,

})

In [6]:
# create custom_toolbox
# num_graph, args_train, stage_pool_path => to define the 'evaluate' function 
toolbox = create_toolbox_for_NSGA_RWNN(num_graph, args_train, stage_pool_path, log_file_name)

In [7]:
"""
4. Algorithms
 For the purpose of completeness we will develop the complete generational algorithm.
"""

POP_SIZE = 2    # population size
NGEN = 4    # number of Generation
CXPB = 0.5    # crossover probability 
MUTPB = 0.5    # mutation probability


# log에 기록할 stats
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

logbook = tools.Logbook()
logbook.header = "gen", "evals", "min", "max", "evals_time", "gen_time"

# population 생성.  (toolbox.population은 creator.Individual n개를 담은 list를 반환. (=> population)
now = datetime.datetime.now()
now_str = now.strftime('%Y-%m-%d %H:%M:%S')
print("Initialion starts ...")
logging.info("Initialion starts at " + now_str)
init_start_time = time.time()

pop = toolbox.population(n=POP_SIZE)

# Evaluate the individuals with an invalid fitness
invalid_ind = [ind for ind in pop if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)    # .evaluate는 tuple을 반환. 따라서 fitnesses는 튜플을 원소로 가지는 list
for ind, fit in zip(invalid_ind, fitnesses):
    ind.fitness.values = fit   # ind.fitness.values = (val_accuracy, flops) 튜플

# This is just to assign the crowding distance to the individuals
# no actual selection is done
pop = toolbox.select(pop, len(pop))

record = stats.compile(pop)
logbook.record(gen=0, evals=len(invalid_ind), **record)
print(logbook.stream)

now = datetime.datetime.now()
now_str = now.strftime('%Y-%m-%d %H:%M:%S')
print("Initialization is finished at", now_str)
logging.info("Initialion is finished at " + now_str)

init_time = time.time() - init_start_time
logging.info("Initialion time = " + str(init_time) + "s")


print()

# Begin the generational process
for gen in range(1, NGEN):
    now = datetime.datetime.now()
    now_str = now.strftime('%Y-%m-%d %H:%M:%S')
    print("#####", gen, "th generation starts at", now_str)
    logging.info(str(gen) + "th generation starts at" + now_str)
    
    start_gen = time.time()
    # Vary the population
    offspring = tools.selTournamentDCD(pop, len(pop))
    offspring = [toolbox.clone(ind) for ind in offspring]

    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if random.random() <= CXPB:
            toolbox.mate(ind1, ind2)

        toolbox.mutate(ind1, indpb=MUTPB)
        toolbox.mutate(ind2, indpb=MUTPB)
        del ind1.fitness.values, ind2.fitness.values

    # Evaluate the individuals with an invalid fitness
    print("##### Evaluation starts")
    start_time = time.time()
    
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    eval_time_for_one_generation = time.time() - start_time        
    print("##### Evaluation ends (Time : %.3f)" % eval_time_for_one_generation)
    
    # Select the next generation population
    pop = toolbox.select(pop + offspring, POP_SIZE)
    
    gen_time = time.time() - start_gen
    print('##### [gen_time: %.3fs]' % gen_time, gen, 'th generation is finished.')
    
    record = stats.compile(pop)
    logbook.record(gen=gen, evals=len(invalid_ind), **record,
                   evals_time=eval_time_for_one_generation, gen_time=gen_time)
    
    logging.info('Gen [%03d/%03d] -- evals: %03d, evals_time: %.4fs, gen_time: %.4fs' % (gen, NGEN, len(invalid_ind), eval_time_for_one_generation, gen_time))
    print(logbook.stream)

Initialion starts ...
	 - Epoch: [0][0/196]	Time 15.846 (15.846)	Loss 6.9489 (6.9489)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
	 - Epoch: [0][100/196]	Time 0.763 (0.936)	Loss 1.7401 (2.0449)	Prec@1 38.672 (29.220)	Prec@5 83.984 (79.649)
Validation_time 18.390 Prec@1 40.810 Prec@5 89.870
	 - Epoch: [1][0/196]	Time 8.796 (8.796)	Loss 1.4197 (1.4197)	Prec@1 50.000 (50.000)	Prec@5 92.188 (92.188)
	 - Epoch: [1][100/196]	Time 0.773 (0.869)	Loss 1.4152 (1.4213)	Prec@1 51.172 (48.298)	Prec@5 92.188 (92.528)
Validation_time 17.918 Prec@1 48.850 Prec@5 93.540
	 - Epoch: [2][0/196]	Time 7.113 (7.113)	Loss 1.2195 (1.2195)	Prec@1 56.250 (56.250)	Prec@5 94.922 (94.922)
	 - Epoch: [2][100/196]	Time 0.783 (0.858)	Loss 0.9407 (1.0767)	Prec@1 64.453 (61.680)	Prec@5 96.875 (96.001)
Validation_time 17.680 Prec@1 66.390 Prec@5 97.150
	 - Epoch: [3][0/196]	Time 7.002 (7.002)	Loss 0.7479 (0.7479)	Prec@1 71.875 (71.875)	Prec@5 97.656 (97.656)
	 - Epoch: [3][100/196]	Time 0.813 (0.850)	Loss 0.8698 (0.8165)	P

	 - Epoch: [0][0/196]	Time 9.442 (9.442)	Loss 6.8902 (6.8902)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.000)
	 - Epoch: [0][100/196]	Time 0.956 (1.058)	Loss 1.6264 (2.0192)	Prec@1 39.844 (29.533)	Prec@5 87.891 (80.299)
Validation_time 21.703 Prec@1 36.030 Prec@5 90.060
	 - Epoch: [1][0/196]	Time 7.751 (7.751)	Loss 1.5032 (1.5032)	Prec@1 42.578 (42.578)	Prec@5 92.969 (92.969)
	 - Epoch: [1][100/196]	Time 1.018 (1.048)	Loss 1.3085 (1.3914)	Prec@1 51.172 (49.520)	Prec@5 92.969 (92.837)
Validation_time 21.038 Prec@1 46.960 Prec@5 91.620
	 - Epoch: [2][0/196]	Time 7.697 (7.697)	Loss 1.0410 (1.0410)	Prec@1 65.234 (65.234)	Prec@5 95.312 (95.312)
	 - Epoch: [2][100/196]	Time 0.943 (1.043)	Loss 1.0889 (1.0596)	Prec@1 61.328 (62.616)	Prec@5 96.484 (96.094)
Validation_time 21.189 Prec@1 56.840 Prec@5 94.940
	 - Epoch: [3][0/196]	Time 7.628 (7.628)	Loss 0.9223 (0.9223)	Prec@1 67.578 (67.578)	Prec@5 97.656 (97.656)
	 - Epoch: [3][100/196]	Time 0.990 (1.036)	Loss 0.8415 (0.8125)	Prec@1 69.531 (71.465)	Pr

ValueError: selTournamentDCD: individuals length must be a multiple of 4

In [ ]:
# Check logbook
logbook

### logbook - plot

In [ ]:
type(logbook)

In [ ]:
len(logbook)

In [ ]:
logbook[0]

In [ ]:
logbook[0]['min']

In [ ]:
-logbook[0]['min'][0], logbook[0]['min'][1]

In [ ]:
min_val_acc = []
min_flops = []

max_val_acc = []
max_flops = []

for i in range(len(logbook)):
    min_val_acc_i, min_flops_i = -logbook[i]['min'][0], logbook[i]['min'][1]
    max_val_acc_i, max_flops_i = -logbook[i]['max'][0], logbook[i]['max'][1]
    
    min_val_acc.append(min_val_acc_i)
    min_flops.append(min_flops_i)
    max_val_acc.append(max_val_acc_i)
    max_flops.append(max_flops_i)    

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# NSGA-2가 제대로 동작함을 알 수 있음.
plt.plot(min_val_acc, min_flops)

plt.xlabel('min_val_acc')
plt.ylabel('min_flops')
plt.title('Experiment Result')

plt.show()

In [ ]:
# gen time 확인
gen_time_list = []

for i in range(1, len(logbook)):
    # 첫번째 initialize 에는 gen_time이 없음
    gen_time_i = logbook[i]['gen_time']
    gen_time_list.append(gen_time_i)

In [ ]:
plt.plot(gen_time_list)

plt.xlabel('generation')
plt.ylabel('gen_time_list')
plt.title('Experiment Result')

plt.show()